<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-中类方法、类实例方法、静态方法有何区别?" data-toc-modified-id="Python-中类方法、类实例方法、静态方法有何区别?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python 中类方法、类实例方法、静态方法有何区别?</a></span></li><li><span><a href="#Python-中如何动态获取和设置对象的属性?" data-toc-modified-id="Python-中如何动态获取和设置对象的属性?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python 中如何动态获取和设置对象的属性?</a></span></li><li><span><a href="#Python-函数调用的时候参数的传递方式是值传递还是引用传递？" data-toc-modified-id="Python-函数调用的时候参数的传递方式是值传递还是引用传递？-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Python 函数调用的时候参数的传递方式是值传递还是引用传递？</a></span></li></ul></div>

## Python 中类方法、类实例方法、静态方法有何区别?   
<br/>  
* `类方法`: 是类对象的方法，在定义时需要在上方使用`@classmethod`进行装饰，形参为`cls`，表示类对象，类对象和实例对象都可调用
* `类实例方法`: 是类实例化对象的方法，只有实例对象可以调用，形参为`self`，指代对象本身
* `静态方法`: 是一个任意函数，在其上方使用`@staticmethod`进行装饰，可以用对象直接调用，静态方法实际上跟该类没有太大关系

## Python 中如何动态获取和设置对象的属性?

```python
class Parent(object):
    
    x = 101                   # 类变量

    def __init__(self):
        self.y = 201          # 实例变量

if __name__ == '__main__':

    if hasattr(Parent, 'x'):
        print getattr(Parent, 'x')       # 101
        setattr(Parent, 'x', 1001)

    print getattr(Parent, 'x')           # 1001        

    p = Parent()
    if hasattr(p, 'y'):
        print getattr(p, 'y')            # 201
        setattr(p, 'y', 2001)

    print getattr(p, 'y')                # 2001
```

## Python 函数调用的时候参数的传递方式是值传递还是引用传递？
[Link1](https://www.cnblogs.com/bingabcd/p/6671368.html)

